In [1]:
from datetime import date
from glob import glob
import json
import math
import os
import sys
import time

import gspread
import numpy as np
import pandas as pd

In [2]:
sys.path.append(r'C:\Users\lesliec\code')

In [3]:
from tbd_eeg.tbd_eeg.data_analysis.eegutils import EEGexp
from tbd_eeg.tbd_eeg.data_analysis.Utilities.utilities import get_events_wdict, qualitycheck_trials, find_nearest_ind
from PCIst.PCIst.pci_st import calc_PCIst, dimensionality_reduction

#### Load excel sheet with metadata for experiments

In [4]:
_gc = gspread.service_account() # need a key file to access the account
_sh = _gc.open('Templeton-log_exp') # open the spreadsheet
_df = pd.DataFrame(_sh.sheet1.get()) # load the first worksheet
metadata = _df.T.set_index(0).T # put it in a nicely formatted dataframe

In [5]:
all_mice = np.unique(metadata['mouse_name'].values)
print(all_mice)

['mouse631037' 'mouse654182' 'mouse655955' 'mouse655956' 'mouse657903'
 'mouse666193' 'mouse666194' 'mouse666196' 'mouse669117' 'mouse669118'
 'mouse673449' 'mouse676726' 'mouse676727' 'mouse678912' 'mouse678913'
 'mouse688277' 'mouse689239' 'mouse689240' 'mouse689241' 'mouse689242'
 'mouse692643' 'mouse692644' 'mouse703062' 'mouse703063' 'mouse703064'
 'mouse703065' 'mouse709399' 'mouse709400' 'mouse709401' 'mouse709402'
 'mouse720762' 'mouse724057' 'mouse728449' 'mouse730911' 'mouse730913'
 'mouse735049' 'mouse735050' 'mouse735051' 'mouse735052']


In [6]:
mousemeta = metadata.iloc[82]
mousemeta

0
mouse_name                                                                            mouse730911
sex (M/F)                                                                                       F
exp_name                                                               aw_ket_2024-05-02_10-41-41
brain states                                                                  awake850/ket+psi850
drug                                                                        ketanserin+psilocybin
stimulation                                                                    electrical/sensory
visual_stim                                                                         natural_scene
audio_stim                                                                                     NA
ISI (sec)                                                                       [6.5 7.5] / [2 3]
stimulus duration (msec)                                                       0.2 / 200,500,1000
Current (uA)      

In [7]:
if mousemeta.visual_stim == 'natural_scene':
    print('Oh yeah, natural scenes, bb.')

Oh yeah, natural scenes, bb.


In [8]:
## Load stim_log ##
rec_folder = os.path.join('F:\psi_exp', mousemeta['mouse_name'], mousemeta['exp_name'], 'experiment1', 'recording1')
print(rec_folder)

F:\psi_exp\mouse730911\aw_ket_2024-05-02_10-41-41\experiment1\recording1


Testing to see if the weird character is in the log.

In [9]:
glob(os.path.join(rec_folder, '*stim_log.csv'), recursive=True)

['F:\\psi_exp\\mouse730911\\aw_ket_2024-05-02_10-41-41\\experiment1\\recording1\\all_stim_log.csv']

In [10]:
if len(glob(os.path.join(rec_folder, '*stim_log.csv'), recursive=True)) == 1:
    stim_file = glob(os.path.join(rec_folder, '*stim_log.csv'), recursive=True)[0]
else:
    print('Could not find one stim_log file')

In [13]:
stim_log = pd.read_csv(stim_file)
print(np.unique(stim_log['stim_type'].values))

['biphasic' 'natural_scene' '\ufeffnatural_scene']


In [17]:
stim_log[stim_log['stim_type'] == '\ufeffnatural_scene']

,stim_type,parameter,duration,onset,offset,sweep,ISI
660,﻿natural_scene,88,1.0,7521.85604,7522.85763,1,2.518012


In [12]:
stim_log_test = pd.read_csv(stim_file, encoding='utf-8-sig')
print(np.unique(stim_log_test['stim_type'].values))

['biphasic' 'natural_scene' '\ufeffnatural_scene']


In [16]:
test_file = os.path.join(rec_folder, 'testlog.csv')
stim_log_test.to_csv(test_file, index=False, encoding='utf-8-sig')

In [17]:
test2 = pd.read_csv(test_file)
print(np.unique(test2['stim_type'].values))

['biphasic' 'natural_scene' '\ufeffnatural_scene']


Testing deleting files

In [11]:
test_file = os.path.join(rec_folder, 'testlog.csv')
if os.path.exists(test_file):
    print('Yes, log exists')

In [10]:
os.remove(test_file)

## Loop through Templeton log to find stim files with weird character

In [23]:
for indi, mousemeta in metadata[79:].iterrows():
    print(f'{mousemeta.mouse_name}: {mousemeta.exp_name}')
    if 'sensory' not in mousemeta['stimulation']:
        print(' No visual stim\n')
        continue
        
    ## FOR LOCAL FILES ##
#     rec_folder = os.path.join('F:\psi_exp', mousemeta['mouse_name'], mousemeta['exp_name'], 'experiment1', 'recording1')
    ## FOR SERVER FILES ##
    rec_folder = os.path.join('P:\\', mousemeta['mouse_name'], mousemeta['exp_name'], 'experiment1', 'recording1')
    if len(glob(os.path.join(rec_folder, '*stim_log.csv'), recursive=True)) == 1:
        stim_file = glob(os.path.join(rec_folder, '*stim_log.csv'), recursive=True)[0]
    else:
        print(' Could not find one stim_log file\n')
        continue
    
    stim_log = pd.read_csv(stim_file)
    if np.any([True if '\ufeff' in x else False for x in np.unique(stim_log['stim_type'].values)]):
        print(' NEED to fix stim log')
        ## For local files ##
        stim_log['stim_type'] = [x.replace('\ufeff', '') for x in stim_log['stim_type'].values]
        stim_log.to_csv(stim_file, index=False)
        print(' *FIXED*')
    else:
        print(' Stim log is OK')
    
    print('')




mouse730913: aw_sal_2024-04-24_10-43-30
 NEED to fix stim log
 *FIXED*

mouse730913: aw_ket_2024-04-26_10-11-34
 NEED to fix stim log
 *FIXED*

mouse730911: aw_sal_2024-05-01_11-57-16
 NEED to fix stim log
 *FIXED*

mouse730911: aw_ket_2024-05-02_10-41-41
 NEED to fix stim log
 *FIXED*

mouse730911: aw_stim_train_2024-05-03_10-33-23
 No visual stim

mouse735049: aw_sal_2024-05-22_11-05-25
 NEED to fix stim log
 *FIXED*

mouse735049: aw_ket_2024-05-23_11-42-08
 NEED to fix stim log
 *FIXED*

mouse735049: aw_stim_train_2024-05-24_10-36-53
 No visual stim

mouse735050: aw_sal_2024-05-29_10-07-49
 Stim log is OK

mouse735050: aw_psi_2024-05-30_09-46-15
 Stim log is OK

mouse735050: aw_sal2_2024-05-31_10-24-49
 Stim log is OK

mouse735051: aw_sal_2024-06-12_11-38-41
 Could not find one stim_log file

mouse735051: aw_psi_2024-06-13_10-44-01
 Could not find one stim_log file

mouse735051: aw_sal2_2024-06-14_10-36-57
 Could not find one stim_log file

mouse735052: aw_sal_2024-06-19_11-07-23
 C